## Traniner API 를 이용한 모델 fine tuning

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

## 학습(Training)

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [3]:
# 모델 정의
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Trainer 정의

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [5]:
trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.4971, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}
{'loss': 0.2731, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}
{'train_runtime': 791.7733, 'train_samples_per_second': 13.898, 'train_steps_per_second': 1.739, 'train_loss': 0.3127076281268439, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3127076281268439, metrics={'train_runtime': 791.7733, 'train_samples_per_second': 13.898, 'train_steps_per_second': 1.739, 'train_loss': 0.3127076281268439, 'epoch': 3.0})

## 평가 (Evaluation)

In [6]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

  0%|          | 0/51 [00:00<?, ?it/s]

(408, 2) (408,)


In [9]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)
preds.shape

(408,)

In [10]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

C:\Users\KSH\AppData\Local\Temp\ipykernel_13972\3131256073.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


{'accuracy': 0.8553921568627451, 'f1': 0.8984509466437176}

In [11]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
# 새 Traner 정의

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.368516743183136, 'eval_accuracy': 0.8480392156862745, 'eval_f1': 0.8864468864468864, 'eval_runtime': 5.0008, 'eval_samples_per_second': 81.587, 'eval_steps_per_second': 10.198, 'epoch': 1.0}
{'loss': 0.5034, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.6127890348434448, 'eval_accuracy': 0.8357843137254902, 'eval_f1': 0.8885191347753744, 'eval_runtime': 12.5273, 'eval_samples_per_second': 32.569, 'eval_steps_per_second': 4.071, 'epoch': 2.0}
{'loss': 0.253, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.7418830990791321, 'eval_accuracy': 0.8578431372549019, 'eval_f1': 0.8982456140350877, 'eval_runtime': 18.0136, 'eval_samples_per_second': 22.65, 'eval_steps_per_second': 2.831, 'epoch': 3.0}
{'train_runtime': 1051.0903, 'train_samples_per_second': 10.469, 'train_steps_per_second': 1.31, 'train_loss': 0.3061262004688847, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3061262004688847, metrics={'train_runtime': 1051.0903, 'train_samples_per_second': 10.469, 'train_steps_per_second': 1.31, 'train_loss': 0.3061262004688847, 'epoch': 3.0})